In [1]:
import numpy as np
def play_games(policy_1, policy_2, env, num_games):
    scores = {agent: 0 for agent in env.possible_agents}
    total_rewards = {agent: 0 for agent in env.possible_agents}
    round_rewards = []
    

    for i in range(num_games):
        print("Game number: ", i)
        rewards = {agent: 0 for agent in env.possible_agents}
        env.reset()

        for agent in env.agent_iter():
            obs, reward, termination, truncation, info = env.last()

            # Separate observation and action mask
            observation, action_mask = obs.values()

            for a in env.agents:
                rewards[a] += env.rewards[a]             
            if termination or truncation:
                winner = max(env.rewards, key=env.rewards.get)
                scores[winner] += 1 # only tracks the largest reward (winner of game)
                for a in env.possible_agents:
                    total_rewards[a] += rewards[a]
                break
            else:
                if agent == env.possible_agents[0]:
                    act = policy_1.compute_single_action(obs)[0]
                else:
                    act = policy_2.compute_single_action(obs)[0]
            env.step(act)
    env.close()
    print("Scores: ", scores)
    print("Total rewards: ", total_rewards)
    return scores, total_rewards

In [2]:
import coup_v2

In [3]:

def env_creator():

    env = coup_v2.env()
    return env

In [12]:
print("hello")

hello


In [7]:
"""Scripts to play all policies in a folder pairwise against each and and record the results in
a pandas dataframe. The results should then be displayed as a heatmap.
"""

import coup_v2
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog
from ray.rllib.examples.models.action_mask_model import TorchActionMaskModel as ActionMaskModel
from models import ActionMaskCentralisedCritic
import ray
from utils import get_experiment_folders, get_checkpoints_folder, get_sorted_checkpoints
import argparse
import pandas as pd
import os
from remove_equals import fix_dir_names
import time

scores_df = pd.DataFrame(columns=["model_1", "model_2", "winrate"])
rewards_df = pd.DataFrame(columns=["model_1", "model_2", "avg_rewards"])


num_games = 100
# use arg parsing to get the local directory to the folder containing the models
#argparser = argparse.ArgumentParser()
#argparser.add_argument("--models_dir", type=str, default="./ray_results/PPO_decentralised", help="Directory containing the models")
main_folder = os.path.abspath("./ray_results/PPO_decentralised")

model_paths = get_experiment_folders(main_folder)
print("model_names")

ray.shutdown()
ray.init(local_mode=True, num_cpus=8, )
ModelCatalog.register_custom_model("am_model", ActionMaskModel)
register_env("Coup", lambda config: PettingZooEnv(env_creator()))
env = coup_v2.env()

for model_1 in range(len(model_paths[:10])):
    for model_2 in range(len(model_paths[:10])):
        
        print(f"Model 1 {model_paths[model_1]}")
        print(f"Model 2 {model_paths[model_2]}")
        # dont want to play the games twice
        if model_1 < model_2 and os.path.exists(model_paths[model_1]) and os.path.exists(model_paths[model_2]):
            
            # load both models using the respective model path
            try:
                model_1_path = get_sorted_checkpoints(get_checkpoints_folder(model_paths[model_1]))[-1]
                model_2_path = get_sorted_checkpoints(get_checkpoints_folder(model_paths[model_2]))[-1]
                

                policy_1 = Algorithm.from_checkpoint(model_1_path).get_policy(policy_id="policy")
                policy_2 = Algorithm.from_checkpoint(model_2_path).get_policy(policy_id="policy")      



                scores, round_reward = play_games(policy_1, policy_2, env, num_games)
                
                del policy_1, policy_2

                temp_scores = pd.DataFrame(columns=["model_1", "model_2", "winrate"])
                temp_scores["model_1"] = [model_paths[model_1]]
                temp_scores["model_2"] = [model_paths[model_2]]
                temp_scores["winrate"] = [scores["player_1"]/num_games]
                scores_df = pd.concat([scores_df, temp_scores], ignore_index=True)
                
                time.sleep(1)
                
            except Exception as e:
                print(e)
                pass
ray.shutdown()
        #print(scores_df)

model_names


2024-04-22 19:22:31,551	INFO worker.py:1752 -- Started a local Ray instance.


Model 1 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00000_0_clip_param-0.1000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-26-53
Model 2 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00000_0_clip_param-0.1000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-26-53
Model 1 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00000_0_clip_param-0.1000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-26-53
Model 2 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00001_1_clip_param-0.2000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-50-44
:job_id:01000000
:actor_name:RolloutWorker


c:\Users\josep\Documents\Coup-RL\env\lib\site-packages\ray\rllib\algorithms\algorithm.py:500: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\josep\Documents\Coup-RL\env\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\josep\Documents\Coup-RL\env\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be rem

:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker


...

:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
System error: Unknown error
Model 1 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00008_8_clip_param-0.3000-gamma-0.9900-train_batch_size-10000_2024-04-22_00-04-34
Model 2 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00000_0_clip_param-0.1000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-26-53
Model 1 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00008_8_clip_param-0.3000-gamma-0.9900-train_batch_size-10000_2024-04-22_00-04-34
Model 2 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00001_1_clip_param-0.2000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-50-44
Model 1 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00008_8_clip_param-0.3000-gamma-0.9900-train_batch_size-10000_2024-04-22_00-04-34
Mode

:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker


:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker


2024-04-22 19:24:03,362	ERROR actor_manager.py:517 -- Ray error, taking actor 1 out of service. System error: Unknown error
2024-04-22 19:24:03,365	ERROR actor_manager.py:517 -- Ray error, taking actor 2 out of service. System error: Unknown error
2024-04-22 19:24:03,366	ERROR actor_manager.py:517 -- Ray error, taking actor 3 out of service. System error: Unknown error


System error: Unknown error
Model 1 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00009_9_clip_param-0.1000-gamma-0.9000-train_batch_size-20000_2024-04-22_00-31-33
Model 2 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00000_0_clip_param-0.1000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-26-53
Model 1 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00009_9_clip_param-0.1000-gamma-0.9000-train_batch_size-20000_2024-04-22_00-31-33
Model 2 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00001_1_clip_param-0.2000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-50-44
Model 1 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\PPO_Coup_1bd85_00009_9_clip_param-0.1000-gamma-0.9000-train_batch_size-20000_2024-04-22_00-31-33
Model 2 c:\Users\josep\Documents\Coup-RL\Coup\coup\ray_results\PPO_decentralised\P

In [11]:
print("hello")

hello


In [ ]:
main_folder

'c:\\Users\\josep\\Documents\\Coup-RL\\Coup\\coup\\ray_results\\PPO_decentralised'

In [ ]:
#from utils import get_last_agent_path
print(model_paths)

#Algorithm.from_checkpoint(model_paths[-6]).get_policy(policy_id="policy")
#model_paths[-30]
ModelCatalog.register_custom_model("am_model", ActionMaskModel)
Algorithm.from_checkpoint(max(model_paths, key=os.path.getmtime)).get_policy(policy_id="policy")

['c:\\Users\\josep\\Documents\\Coup-RL\\Coup\\coup\\ray_results\\PPO_decentralised\\PPO_Coup_1bd85_00000_0_clip_param-0.1000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-26-53', 'c:\\Users\\josep\\Documents\\Coup-RL\\Coup\\coup\\ray_results\\PPO_decentralised\\PPO_Coup_1bd85_00001_1_clip_param-0.2000-gamma-0.9000-train_batch_size-10000_2024-04-21_20-50-44', 'c:\\Users\\josep\\Documents\\Coup-RL\\Coup\\coup\\ray_results\\PPO_decentralised\\PPO_Coup_1bd85_00002_2_clip_param-0.3000-gamma-0.9000-train_batch_size-10000_2024-04-21_21-28-22', 'c:\\Users\\josep\\Documents\\Coup-RL\\Coup\\coup\\ray_results\\PPO_decentralised\\PPO_Coup_1bd85_00003_3_clip_param-0.1000-gamma-0.9500-train_batch_size-10000_2024-04-21_21-54-16', 'c:\\Users\\josep\\Documents\\Coup-RL\\Coup\\coup\\ray_results\\PPO_decentralised\\PPO_Coup_1bd85_00004_4_clip_param-0.2000-gamma-0.9500-train_batch_size-10000_2024-04-21_22-20-05', 'c:\\Users\\josep\\Documents\\Coup-RL\\Coup\\coup\\ray_results\\PPO_decentralised\\PPO_Co

ValueError: Given checkpoint does not seem to be valid! No file with the name `algorithm_state.[pkl|msgpck]` (or `checkpoint-[0-9]+`) found.

In [ ]:
Algorithm.from_checkpoint("./ray_results/PPO_decentralised/PPO_decentralised_0_2021-08-02_14-00-00q1v1q1v1/checkpoint_000001/checkpoint-1").get_policy(policy_id="policy")

ValueError: Given checkpoint (./ray_results/PPO_decentralised/PPO_decentralised_0_2021-08-02_14-00-00q1v1q1v1/checkpoint_000001/checkpoint-1) not found! Must be a checkpoint directory (or a file for older checkpoint versions).